In [ ]:
target='manual'
run='classic'
titles = [
    'Star Trek (2009)', 
    'Star Trek: First Contact (1996)', 
#     'Star Trek: Generations (1994)', 
#     'Star Trek II: The Wrath of Khan (1982)', 'Star Trek IV: The Voyage Home (1986)', 
#     'Star Trek Into Darkness (2013)', 'Star Trek VI: The Undiscovered Country (1991)', 
#     'Star Trek III: The Search for Spock (1984)', 
#     'Star Trek: The Motion Picture (1979)', 
#     'Star Trek: Insurrection (1998)', 'Star Trek V: The Final Frontier (1989)', 
#     'Star Trek Beyond (2016)', 'Star Trek: Nemesis (2002)', 'Star Trek: Renegades (2015)', 
#     'Star Trek: Of Gods and Men (2007)'
]

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") 
sys.path.append("../es/") 


import random
myseed = 13
random.seed(myseed)

import pandas as pd
import numpy as np
import es
import es.data_config
from es.data.movielens_data import *
import es.runner.scenario_runner
from es.runner.view_models import show_cube_grid, RankSquareViewModel, SpectrumViewModel
import es.elections
import es.utils
import es.scorers
# from es import *
from es.metrics import *
import seaborn as sns

import bokeh
import bokeh.plotting
from bokeh.io import output_notebook
output_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

from pandasql import sqldf
pysqldf = lambda q, df: sqldf(q, dict(tmp=df))

pd.set_option('display.max_rows', 40)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


## Calculate the metrics for a product_RE
### Star Trek Calibration Idea
When searching for star trek, the other movies from the series should be visible

NOTE: BM25 calibration starts with simulating the full 2 dimensional grid of `b` and `k` parameters. Then we notice that the maximum average `top10count` is achieved by multiple `b` values. For simplicity and clearer visualisation we choose one of those `b=25` and allow the optimal `k` parameter be selected.

In general, although BM25 fares slightly better on our top10count Star Trek calibration than TFIDF, we choose to use TFIDF for its simplicity.

## Calculate calibration values

In [ ]:
%%time
import numpy as np
metrics_list=[]
# bm_b_list=[round(0.0+x*.1,1) for x in np.linspace(11)]
# b in [0,1], b=1 means the document length is NOT taken into account
# k is proportional to the weight on TF (rather than IDF)
# metrics_list+=[dict(kind='bm25', k=k, b=b) for k in np.linspace(25, 25, 1) for b in bm_b_list]

tf_base_list=[round(1.2+x*.2,1) for x in range(9)]  # for GAIW
metrics_list+=[dict(kind='tfxtoidf', base=base) for base in tf_base_list] # for GAIW

# metrics_list=[dict(kind='bm25', k=k, b=b) for k in [20, 21, 22, 23] for b in [1.0]]
# metrics_list=[dict(kind='tfxtoidf', base=b) for b in [1.95, 2.0]]

out_folder=f'../out/{target}/movielens-StarTrek-calibration/{run}'
if not os.path.exists(out_folder):
    os.makedirs(out_folder)


titles = [t.replace('(', '.').replace(')', '.') for t in titles]

datasets = [
    es.data_config.data_movielens_25m,  # for GAIW
#     es.data_config.data_movielens_1m,
]

def startrek_content(df):
    return len(pysqldf("select product_name from tmp where product_name like '%Star Trek%'", df))

rows=[]

for dataset in datasets:
    dataset_iframes = es.elections.InstacartDataFrames.from_data(MovielensData(dataset), min_popularity=20) # for GAIW
    dataset_utils = es.electionutils.ElectionUtils(dataset_iframes)
    
#     dataset_utils.products_tf_metric_grid(
#         titles, 
#         ['tf'], 
#         hue='order_count',
#         metrics_list=metrics_list
#     )

    for title in titles:
        print(title)
        movies_df = dataset_utils.find_products(title, min_order_count=1)
#         print(f"processing {movies_df}")
        found_df = dataset_utils.calc_found_df(movies_df, metrics_list=metrics_list)
        for m in metrics_list:
            metric = Metric.from_dict(m).get_name()
            df = found_df.sort_values(metric, ascending=False)
            title_tf = df.tf.max()
            df = df[:10]
            row=dict(title=title+f" (tf={title_tf})", dataset=dataset[-4:], metric=metric, top10count=startrek_content(df), title_tf=title_tf)
            rows.append(row)

df = pd.DataFrame(rows)
# df

In [ ]:
metric_converter=lambda m: f"$TFIDF, \gamma$ = {m[2:5].replace('_', '.')}" if 'tf' in m else m

sns.set(font_scale=1.4)   # for GAIW
dataset_utils.products_tf_metric_grid(
#     titles,     
    [
        'Star Trek III: The Search for Spock .1984.', # for GAIW
        'Star Trek: Renegades .2015.'  # for GAIW
    ],
    ['TF'], 
    # hue='order_count',
#     metrics_list=metrics_list,
    metrics_list=[m for m in metrics_list if round(m['base'],1) in [1.2, 1.6, 2.0, 2.4, 2.8]],
    metric_kind='TFIDF',
    metric_converter=metric_converter,
    out_folder=out_folder
)

## Calibration visualisation

In [ ]:
df['title_notf']=df.title.apply(lambda x: x.split('.')[0])

def plot_top10counts(metric, metric_df, xticks, title, xlabel):
    sorted_metric_df = metric_df.sort_values(['metric', 'title_tf'])
    g = sns.catplot(x="metric", y="top10count", 
                    hue="title_tf", 
                    col='dataset',
                    palette=sns.cubehelix_palette(start=0.5, rot=0.8, dark=.0, light=.9, n_colors=15),
#                     palette=sns.color_palette("light:b", 15),
#                     palette=sns.color_palette("bright", 15),
    #             markers=["^", "o"], linestyles=["--", "-"],
                kind="strip", data=sorted_metric_df, aspect=3,
                    legend=True,
                    s=8, 
#                     dodge=True,
#                     jitter=True
                   )
    (
    g.set_axis_labels(xlabel, "the count of Star Trek movies\nin the top 10")
      .set_xticklabels(xticks)
#       .set_yticklabels(range(0,11))
      .set_titles(title)
    #   .set_titles("{col_name} {col_var}")
    #   .set(ylim=(0, 1))
      .despine(offset=10)
    #   .set_xticklabels(rotation=10)
    )
    g._legend.set_title("  movie\napprovals")
    
    fig_filename = f"{out_folder}/calib_top10_{metric}.png"
    print(f"Saving balance figure to '{fig_filename}'")
    g.savefig(fig_filename)
    
#     new_labels = [t for t in pysqldf("select title from tmp group by title_tf, title", metric_df).title]
#     for t, l in zip(g._legend.texts, new_labels): t.set_text(l)    

sns.set(font_scale=1.4, style='white') 
# plot_top10counts('bm25', pysqldf("select * from tmp where metric like '%bm%'", df), bm_b_list, "BM25 $b$ calibration", "$b$")
plot_top10counts('tfidf', pysqldf("select * from tmp where metric like '%tf%'", df), tf_base_list, "TFIDF $\gamma$ calibration", "$\gamma$") # for GAIW


# df

## The calibration average values

In [ ]:
df['gamma']=df.metric.apply(lambda x: float(x[2:5].replace("_", ".")) if 'tf' in x else None)
df['b']=df.metric.apply(lambda x: float(x[6:10].replace("_", ".")) if 'bm' in x else None)
df['k']=df.metric.apply(lambda x: float(x[12:15].replace("_", ".")) if 'bm' in x else None)
df

# print(pysqldf("select b, k, avg(top10count) from tmp where metric like '%bm%' group by metric order by metric asc", df)[:10])
calib_top10_avg_df = pysqldf("select metric, avg(top10count) from tmp where metric like '%tf%' group by metric order by metric asc", df)[:10]
calib_top10_avg_df.to_csv(f"{out_folder}/calib_top10_tfidf_avg.csv")
calib_top10_avg_df


### Star Trek test ^^:
The higher the line across the movies the better the metric.

- for each dataset (x axis)
- take a star trek movie (one graph per movie),
- reduce the elections to search for it, 
- calculate various metrics (`hues` on the graph)
- find the top10 movies according to the metric
- calculate the count of other Star Trek movies in these top 10 (y axis - `top10count`)

## All Star Trek movies with the number of approvals

In [ ]:
print(pysqldf("select distinct title_notf Movie, title_tf 'Approvals' from tmp order by title_tf", df))

## The generated values

In [ ]:
df